In [ ]:
import os
import string
from collections import defaultdict, Counter
from itertools import combinations
%pylab
%matplotlib inline

In [ ]:
%%bash
mkdir -p data
wget -q -nc -P data/ https://raw.githubusercontent.com/cschaffner/InformationTheory/master/Problems/HW1/Alice_eng.txt
wget -q -nc -P data/ https://raw.githubusercontent.com/cschaffner/InformationTheory/master/Problems/HW1/Alice_esp.txt
wget -q -nc -P data/ https://raw.githubusercontent.com/cschaffner/InformationTheory/master/Problems/HW1/Alice_fin.txt
wget -q -nc -P data/ https://raw.githubusercontent.com/cschaffner/InformationTheory/master/Problems/HW1/Alice_ger.txt
wget -q -nc -P data/ https://raw.githubusercontent.com/cschaffner/InformationTheory/master/Problems/HW1/Alice_ita.txt
wget -q -nc -P data/ https://raw.githubusercontent.com/cschaffner/InformationTheory/master/Problems/HW1/permuted_cipher.txt

# Utilities

In [ ]:
DATA_FOLDER="data"
LANGS = ('eng', 'esp', 'fin', 'ger', 'ita')
PATH_TEMPLATE = os.path.join(DATA_FOLDER, "Alice_{}.txt")
SYMBOLS = string.ascii_lowercase

translation = defaultdict(lambda: None)
translation.update({ord(s): ord(s) for s in SYMBOLS})


def read_file(filename):
    """Open file in memory and close fd."""
    with open(filename,'r') as f:
        return f.read()


def preprocess_text(text, t=translation):
    """After converting everything in lowercase replace
    all non letters with empty characters.
    """
    return text.lower().translate(t)


def get_data(file_path):
    """From a file path, return the preprocessed text
    as required from the assignment.
    """
    text = read_file(file_path)
    return preprocess_text(text)


def get_freq(data, labels, relative=True):
    """Given an iterator, returns the frequencies of the symbols 
    in sorted() order.
    """
    freq = {l: 0 for l in labels}
    for d in data:
        try:
            freq[d] += 1
        except KeyError:
            raise RuntimeError(f"Found {d} not existing in the list of symbols: {SYMBOLS}")
            
    labels, freq = zip(*sorted(freq.items()))
    freq = np.array(freq)
    if relative:
        freq = freq / freq.sum()
    return labels, freq


def plot_counter(labels, freq, ax=None):
    indexes = np.arange(len(labels))
    width = 0.8
    if ax is None:
        ax = plt.gca()
    ax.bar(indexes, freq, width, alpha=0.8)
    ax.set_xticks(indexes)
    ax.set_xticklabels(labels)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.set_ylim(0, 0.2)


# Prepare and show data

In [ ]:

# Collections of multiple pre-processed translations of the novel 
novel = {l: get_data(PATH_TEMPLATE.format(l)) for l in LANGS}
novel['cipher'] = get_data(os.path.join(DATA_FOLDER, "permuted_cipher.txt"))


# For each language, contains the relative frequency
freq = {l: get_freq(data, SYMBOLS)[1] for l, data in novel.items()}


# Examples
print("Extract from each of the languages:\n")
for l, t in novel.items():
    print(f"{l:8}: {t[2000:2100]}...")
print("\n")

fig, axes = plt.subplots(2, 3, figsize=(15,7))
for i, (l, data) in enumerate(novel.items()):
    r = i // 3
    c = i % 3
    ax = axes[r, c]
    labels, f = get_freq(data, SYMBOLS)
    plot_counter(labels, f, ax=ax)
    ax.set_title(l)
    
fig.tight_layout(rect=(0.1, 0.1, 0.9, 0.9))
fig.suptitle("Novel alphabet frequencies by language and\nfor the permuted cipher.", fontsize=18)
plt.savefig("relative-freq.eps")
plt.show()



# Compute all pairwise variational distances

Where we have that the total variational distance between $P$ and $Q$ is defined as:

\begin{equation}
||P - Q|| := \frac 1 2 \sum_{x \in \mathcal{X}}| P(x) - Q(x) |
\end{equation}

In [ ]:

def variational_distance(P, Q):
    assert(P.sum() == 1 and Q.sum() == 1)
    return np.abs(P - Q).sum() / 2


def test_variational_distance(freq):
    for l, P in freq.items():
        assert(variational_distance(P, P) == 0)


def print_pairwise_variational_distances(freq):
    pairs = list(combinations((freq.keys()), 2))
    results = {}
    for Pl, Ql in pairs:
        results[(Pl, Ql)] = variational_distance(freq[Pl], freq[Ql])
    
    results = list(results.items())
    results.sort(key=lambda x: x[1], reverse=True)
    for (pl, ql), v in results:
        print(f'{pl}-{ql:8}: {v:.2f}')

test_variational_distance(freq)
print_pairwise_variational_distances(freq)

# Compute the five collision probabilities

The collision probability of a distribution $P$ is defined as:

\begin{equation}
\operatorname{Call}(P) := \sum_{x\in \mathcal{X}} P(x)
\end{equation}

In [ ]:

def collision_probability(P):
    return np.square(P).sum()


def print_collision_probabilities(freq):
    rd = {}
    for l, P in freq.items():
        rd[l] = collision_probability(P)
    
    results = list(rd.items())
    results.sort(key=lambda x: x[1], reverse=True)

    print("Collision probabilities:")
    for l, v in results:
        print(f"{l:8}: {v}")
    
    return rd

rest = print_collision_probabilities(freq)